In this notebook I'm using the OpenPipe client to capture a set of calls to the OpenAI API.

For this example I'll blithely throw engineering best practices to the wind and use the notebook itself to manage dependencies. 😁


In [1]:
%%capture
%pip install openpipe==3.0.3 python-dotenv==1.0.0 datasets==2.14.4

When working with remote datasets (or any data, really), it's a good idea to visually inspect some samples to make sure it looks like you expect. I'll print a recipe.


In [2]:
from datasets import load_dataset

recipes = load_dataset("corbt/unlabeled-recipes")["train"]
print("Recipe dataset shape:\n------------------")
display(recipes)
print("First recipe:\n------------------", recipes["recipe"][0])


Recipe dataset shape:
------------------


Dataset({
    features: ['recipe'],
    num_rows: 5000
})

First recipe:
------------------ Shrimp Creole

Ingredients:
- 20 shrimp (8 oz.)
- 2 c. (16 oz. can) tomato sauce
- 1 small onion, chopped
- 1 celery stalk, chopped
- 1/4 green bell pepper, diced
- 1/4 c. sliced mushrooms
- 3 Tbsp. parsley
- 1/2 tsp. pepper
- 1 to 1-1/2 c. brown rice, prepared according to pkg. directions (not included in exchanges)

Directions:
- Peel, devein and wash shrimp; set aside.
- (If shrimp are frozen, let thaw first in the refrigerator.)
- Simmer tomato sauce, onion, celery, green pepper, mushrooms, parsley and pepper in skillet for 30 minutes.
- Add shrimp and cook 10 to 15 minutes more, until shrimp are tender.
- Serve over brown rice.
- Serves 2.


Mm, delicious. Anyway, we need to generate a training dataset. We'll call GPT-4 on each of our examples.

In this case, I'll ask GPT-4 to classify each recipe along 5 dimensions:

- has_non_fish_meat
- requires_oven
- requires_stove
- cook_time_over_30_mins
- main_dish

That looks like a pretty random list, but there's actually an important unifying thread: I'm looking for meals that my pescatarian brother/co-founder can make in his kitchen-less, near-window-less basement apartment in San Francisco! (If you haven't tried to get an apartment in SF you probably think I'm joking 😂.)

I'll use [OpenPipe](https://github.com/openpipe/openpipe) to track the API calls and form a training dataset. To follow along you'll need to create a free OpenPipe account, then copy your API key from https://app.openpipe.ai/settings into a file called `.env`. You can see an example in [./.env.example](./.env.example).


In [3]:
from openpipe import openai, configure_openpipe
import json
import os
import dotenv

# Use `dotenv` to load the contents of the `.env` file into the environment
dotenv.load_dotenv()

# Configure OpenPipe using the API key from the environment
configure_openpipe(api_key=os.environ["OPENPIPE_API_KEY"])

# Configure OpenAI using the API key from the environment
openai.api_key = os.environ["OPENAI_API_KEY"]


def classify_recipe(recipe: str):
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "Your goal is to classify a recipe along several dimensions.Pay attention to the instructions.",
            },
            {
                "role": "user",
                "content": recipe,
            },
        ],
        functions=[
            {
                "name": "classify",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "has_non_fish_meat": {
                            "type": "boolean",
                            "description": "True if the recipe contains any meat or meat products (eg. chicken broth) besides fish",
                        },
                        "requires_oven": {
                            "type": "boolean",
                            "description": "True if the recipe requires an oven",
                        },
                        "requires_stove": {
                            "type": "boolean",
                            "description": "True if the recipe requires a stove",
                        },
                        "cook_time_over_30_mins": {
                            "type": "boolean",
                            "description": "True if the recipe takes over 30 minutes to prepare and cook, including waiting time",
                        },
                        "main_dish": {
                            "type": "boolean",
                            "description": "True if the recipe can be served as a main dish",
                        },
                    },
                    "required": [
                        "has_non_fish_meat",
                        "requires_oven",
                        "requires_stove",
                        "cook_time_over_30_mins",
                        "main_dish",
                    ],
                },
            }
        ],
        function_call={
            "name": "classify",
        },
        openpipe={"tags": {"prompt_id": "classify-recipe"}, "cache": True},
    )
    return json.loads(completion.choices[0].message.function_call.arguments)


print("Classifying first recipe:\n------------------")
print(classify_recipe(recipes["recipe"][0]))


Classifying first recipe:
------------------
{'has_non_fish_meat': False, 'requires_oven': False, 'requires_stove': True, 'cook_time_over_30_mins': True, 'main_dish': True}


That's working, so I'll go ahead and classify all 5000 recipes with GPT-4. Using GPT-4 for this is slowwww and costs about $40. The model I'm fine-tuning will be much faster -- we'll see if we can make it as good!


In [4]:
for i, recipe in enumerate(recipes["recipe"]):
    if i % 100 == 0:
        recipe_title = recipe.split("\n")[0]
        print(f"Classifying recipe {i}/{len(recipes)}: {recipe_title}")
    try:
        classify_recipe(recipe)
    except Exception as e:
        print(f"Error classifying recipe {i}: {e}")


Classifying recipe 0/5000: Shrimp Creole
Classifying recipe 100/5000: Spoon Bread
Classifying recipe 200/5000: Quadrangle Grille'S Pumpkin-Walnut Cheesecake
Classifying recipe 300/5000: Broccoli Casserole
Classifying recipe 400/5000: Paal Payasam (3-Ingredient Rice Pudding)
Classifying recipe 500/5000: Dirt Dessert
Classifying recipe 600/5000: Dolma, Stuffed Dried Peppers And Eggplants
Classifying recipe 700/5000: Party Pecan Pies
Classifying recipe 800/5000: Pie Crust
Classifying recipe 900/5000: Russian Dressing(Salad Dressing)  
Classifying recipe 1000/5000: O'Brien Potatoes
Classifying recipe 1100/5000: Monster Cookies
Classifying recipe 1200/5000: Striped Fruit Pops
Classifying recipe 1300/5000: Cute Heart-Shaped Fried Egg
Classifying recipe 1400/5000: Steak Marinade
Classifying recipe 1500/5000: Bbq Sauce For Fish Recipe
Classifying recipe 1600/5000: Barbecue Ranch Salad
Classifying recipe 1700/5000: White Fudge
Classifying recipe 1800/5000: Seaton Chocolate Chip Cookies
Classify

Ok, now that my recipes are classified I'll download the training data.

Next up I'll train the model -- check out [./2-train.ipynb](./2-train.ipynb) for details! Just go to https://app.openpipe.ai/request-logs, select all the logs you created, and click "Export". The default 10% testing split is fine for this dataset size.

You'll get two files from that: `train.jsonl` and `test.jsonl`. Move both of them into this repository under `./data/`.
